In [109]:
import os
os.getcwd()
os.chdir('d:\\bot_telegram_customer_support\\sheets')


import utils
import pandas as pd

In [110]:
# Connection To DB
import pyodbc

connection = pyodbc.connect('Driver={SQL Server};'
                            'Server=.;'
                            'Database=MassPanAdmindb;'
                            'Trusted_Connection=yes;')

In [111]:
# Obtaining Google Sheet
sh = utils.getGoogleSpreadsheet(
    'products.json',
    utils.PRODUCTS_SHEET_KEY
)

In [112]:
# Obtener todas las instancias de la DB y añadirla a la Sheet
query = pd.read_sql_query(
    "SELECT [CodInst],[Descrip] FROM [MassPanAdmindb].[dbo].[SAINSTA]", connection
)
instance_df = pd.DataFrame(query)

instances = utils.getOrCreateGoogleWorksheet(sh, 'Instances', cols=2, rows=16)
#utils.writeDataframeInSheet(instance_df, instances, dont_repeat_on='Descrip')

In [113]:
# Obtener todos los productos con  sus datos y añadirlo a la Google Sheet
query = pd.read_sql_query(
    """
    SELECT  prod.CodProd,
		prod.Descrip, 
		prod.CodInst, 
		prod.Marca, 
		prod.Existen, 
		prod.Activo,
		usd.Costo, 
		usd.Mayor, 
		usd.Detal, 
		usd.Especial 
    FROM [MassPanAdmindb].[dbo].[SAPROD] as prod 
    INNER JOIN [MassPanAdmindb].[dbo].[SAPROD_01] AS usd 
    ON (prod.CodProd = usd.CodProd)
    """,
    connection
)
products_df = pd.DataFrame(query)
# Filtrando que los productos esten activos
products_df = products_df.loc[products_df['Activo'] == 1]
# Filtrando que su precio no sea cero
products_df = products_df.loc[products_df['Costo'] > 0]
# Filtrando que los productos no esten en una categoria privada
products_df = products_df.loc[
    (products_df['CodInst'] != 15) & 
    (products_df['CodInst'] != 1025)
    ]
#products = utils.getOrCreateGoogleWorksheet(sh, 'Products', cols=10, rows=100)
#utils.writeDataframeInSheet(products_df, products,dont_repeat_on='CodProd')
products_df

,CodProd,Descrip,CodInst,Marca,Existen,Activo,Costo,Mayor,Detal,Especial
0,00005,CREMOSITO CHEDDAR 220 GR DERILAC,9,DERILAC,1.0,1,1.70,1.85,2.21,2.21
1,0001002,MASA FACIL MI GUSTO 900 GR,11,MI GUSTO,27.0,1,1.50,1.73,1.95,1.95
2,0004,CREMA DE LECHE YARACAL 520GR,9,,5.0,1,1.56,1.69,2.03,2.03
3,000859,PASTA RENATA 500 GR,11,None,0.0,1,0.73,0.95,0.95,0.95
4,000893,SALSA DE TOMATE CON ALBAHACA OLE 340 GR,11,None,0.0,1,0.79,1.03,1.03,1.03
...,...,...,...,...,...,...,...,...,...,...
541,YO018,ADEREZO DE CILANTRO NATUGURT,9,NATUGURT,1.0,1,1.20,1.40,1.56,1.56
542,YO021,ADEREZO DE PIMENTON NATUGURT,9,NATUGURT,1.0,1,1.20,1.40,1.56,1.56
543,YOUGURTCASERO,YOGURT CASERO PEQUEÑO,9,,11.0,1,1.00,1.30,1.30,1.30
544,YOUGURTCASEROG,YOGURT CASERO GRANDE,9,,5.0,1,2.30,3.00,3.00,3.00


In [114]:
query = pd.read_sql_query(
"""
SELECT [CodAlte]
      ,[CodProd]
FROM [MassPanAdmindb].[dbo].[SACODBAR]
""",
connection
)
code_df = pd.DataFrame(query)
code_df
codes = utils.getOrCreateGoogleWorksheet(sh, 'Barcodes', cols=2)